In [1]:
import os
import sys
import numpy as np
import healpy as hp
import histlite as hl
import csky as cy
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib.colors as colors

import seaborn as sns
sns.set_theme(style="whitegrid")

%matplotlib inline
# %matplotlib notebook

from glob import glob
timer = cy.timing.Timer()
time = timer.time

sys.path.append('../../')
from greco_grb.scripts import SETTING
paths = SETTING.PATH()
print(paths)
USER = paths.USER
ICDATA_DIR = paths.ICDATA_DIR
DATA_DIR = paths.DATA_DIR
ANA_DIR = paths.ANA_DIR

from greco_grb.scripts.utils import *


#### Working at GT ####

LOCATION: 	 GT
USER: 	 cchen641
ICDATA_DIR: 	 /storage/home/hhive1/cchen641/data/icecube/data/analyses/online_GRECO/candidate-002-p10
DATA_DIR: 	 /storage/home/hhive1/cchen641/data/icecube/data/greco_grb/data
ANA_DIR: 	 /storage/home/hhive1/cchen641/data/icecube/data/greco_grb/data/csky_output



In [2]:
print("\n===== Setting up csky =====\n")
data_dir = ICDATA_DIR
data_filenames = sorted(glob(data_dir + '/IC86_20*.data.npy'))
sig_filenames = sorted(glob(data_dir + '/IC86_2012.nu*_merged.npy'))
grl_filenames = sorted(glob(data_dir + '/GRL/IC86_20*.data.npy'))

################ energy lower bound #############
min_log_e = np.log10(10)
#################################################
bins_sindec = np.linspace(-1, 1, 25+1)  
bins_logenergy = np.linspace(min_log_e, 4, 25+1)

data = [np.load(data_filename) for data_filename in data_filenames]
data = np.hstack(data)
sig = [np.load(sig_filename) for sig_filename in sig_filenames]
sig = np.hstack(sig)
grl = [np.load(grl_filename) for grl_filename in grl_filenames]
grl = np.hstack(grl)
if min_log_e is not None:
    data_mask = data['logE'] > min_log_e
    data = data[data_mask]
    sig_mask = sig['logE'] > min_log_e
    sig = sig[sig_mask]
    
dataset_spec = cy.selections.CustomDataSpecs.CustomDataSpec(data, sig, np.sum(grl['livetime']),
                                                     sindec_bins=bins_sindec,
                                                     logenergy_bins=bins_logenergy,
                                                     grl=grl, key='greco_v2.10', cascades=True)

ANA_DIR = cy.utils.ensure_dir(ANA_DIR)
# on OSG
# ana_dir = "./"
ana = cy.get_analysis(cy.selections.repo
                      , dataset_spec
                      , dir=ANA_DIR
                      , load_sig=True)  # to save memory: use False





===== Setting up csky =====

Setting up Analysis for:
greco_v2.10
Setting up greco_v2.10...
Energy PDF Ratio Model...
  * gamma = 4.0000 ...
Signal Acceptance Model...
  * gamma = 4.0000 ...
Done.


# 1. Non-injection Binomial Test

In [93]:
def get_post_p(grb_name, pre_p):
    """
    effective trial correction due to multiple time windows
    
    parameters:
        grb_name: str
            name of the grb
        pre_p: float or array_like
            one or multiple pre_trial p value(s)
        
    return:
        post_p: float or array_like
            one or multiple post_trial p value(s)
    """
    pre_post = np.load(ANA_DIR + f"/effective_trial/{grb_name}_effective_trial.npy")
    print(pre_post[:])
    
    idx = np.searchsorted(pre_post['pre_trial_p'], pre_p, side='right')
    idx = np.maximum(idx, 0)
    idx = np.minimum(idx, pre_post['post_trial_p'].shape[0] - 1)
    return idx, pre_post['pre_trial_p'][idx], pre_post['post_trial_p'][idx]
    
    

In [94]:
get_post_p(grb_name="GRB180423A", pre_p=0.99e-6)

[(1.00000e-06, 2.00000e-06) (3.00000e-06, 4.00000e-06)
 (5.00000e-06, 6.00000e-06) ... (3.67733e-01, 3.67734e-01)
 (3.67735e-01, 3.67736e-01) (1.00000e+00, 1.00000e+00)]


(0, 1e-06, 2e-06)

In [85]:
get_post_p(grb_name="GRB180423A", pre_p=1.0e-6)

[0.367719 0.367721 0.367723 0.367725 0.367727 0.367729 0.367731 0.367733
 0.367735 1.      ]


(1, 3e-06, 4e-06)

In [86]:
get_post_p(grb_name="GRB180423A", pre_p=1)

[0.367719 0.367721 0.367723 0.367725 0.367727 0.367729 0.367731 0.367733
 0.367735 1.      ]


(84578, 1.2, 6e-06)

In [73]:
grb_name="GRB180423A"
pre_post = np.load(ANA_DIR + f"/effective_trial/{grb_name}_effective_trial.npy")

In [76]:
pre_post['pre_trial_p'].shape

(169157,)